In [1]:
from data.t5_dataset import T5CursorDataset

dataset = T5CursorDataset('./data/t5.2021.06.07.mat')


from snel_toolkit import analysis

In [3]:
dataset.data

signal_type            targetSize cursorSize blockNums   cursorPos            \
channel                      size       size         n           x         y   
clock_time                                                                     
0 days 00:00:00              57.5         25        14    0.000000  0.000000   
0 days 00:00:00.001000       57.5         25        14    0.000000  0.000000   
0 days 00:00:00.002000       57.5         25        14    0.000000  0.000000   
0 days 00:00:00.003000       57.5         25        14    0.000000  0.000000   
0 days 00:00:00.004000       57.5         25        14    0.000000  0.000000   
...                           ...        ...       ...         ...       ...   
0 days 00:18:35.382000       85.0         25        17 -364.780579 -9.742271   
0 days 00:18:35.383000       85.0         25        17 -365.002655 -9.565752   
0 days 00:18:35.384000       85.0         25        17 -365.224701 -9.389242   
0 days 00:18:35.385000       85.0         25        17 -365.446747 -9.212741   
0 days 00:18:35.386000       85.0         25        17 -365.668793 -9.036249   

signal_type            targetPos    onTarget    decVel            ... spikes  \
channel                        x  y    state         x         y  ...  ch182   
clock_time                                                        ...          
0 days 00:00:00                0  0        1  0.000000  0.000000  ...      0   
0 days 00:00:00.001000         0  0        1  0.000000  0.000000  ...      0   
0 days 00:00:00.002000         0  0        1  0.000000  0.000000  ...      0   
0 days 00:00:00.003000         0  0        1  0.000000  0.000000  ...      0   
0 days 00:00:00.004000         0  0        1  0.000000  0.000000  ...      0   
...                          ... ..      ...       ...       ...  ...    ...   
0 days 00:18:35.382000      -409  0        1 -0.000049  0.000039  ...      0   
0 days 00:18:35.383000      -409  0        1 -0.000049  0.000039  ...      0   
0 days 00:18:35.384000      -409  0        1 -0.000049  0.000039  ...      0   
0 days 00:18:35.385000      -409  0        1 -0.000049  0.000039  ...      0   
0 days 00:18:35.386000      -409  0        1 -0.000049  0.000039  ...      0   

signal_type                                                                   
channel                ch183 ch184 ch185 ch186 ch187 ch188 ch189 ch190 ch191  
clock_time                                                                    
0 days 00:00:00            0     0     0     0     0     0     0     0     0  
0 days 00:00:00.001000     0     0     0     0     0     0     0     0     0  
0 days 00:00:00.002000     0     0     0     0     0     0     0     0     0  
0 days 00:00:00.003000     0     0     0     0     0     0     0     0     0  
0 days 00:00:00.004000     0     0     0     0     0     0     0     0     0  
...                      ...   ...   ...   ...   ...   ...   ...   ...   ...  
0 days 00:18:35.382000     0     0     0     0     0     0     0     0     0  
0 days 00:18:35.383000     0     0     0     0     0     0     0     0     0  
0 days 00:18:35.384000     0     0     0     0     0     0     0     0     0  
0 days 00:18:35.385000     0     0     0     0     0     0     0     0     0  
0 days 00:18:35.386000     0     0     0     0     0     0     0     0     0  

[1115387 rows x 203 columns]